In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from tqdm import tqdm

class Config:
    data_root = 'ml-intensive-yandex-academy-spring-2025/human_poses_data'
    img_train_dir = os.path.join(data_root, 'img_train')
    categories_file = os.path.join(data_root, 'activity_categories.csv')
    train_answers_file = os.path.join(data_root, 'train_answers.csv')
    img_size = (224, 224)
    batch_size = 32
    num_workers = 0
    test_size = 0.15
    val_size = 0.15
    random_state = 42
    num_classes = 20
    lr = 0.001
    weight_decay = 1e-4
    epochs = 50
    flip_prob = 0.5
    save_dir = 'saved_models_densenet'
    growth_rate = 12
    block_config = (4, 4, 4)
    bn_size = 4
    drop_rate = 0.2

class DenseLayer(nn.Module):
    def __init__(self, in_channels, growth_rate, bn_size, drop_rate):
        super().__init__()
        self.norm1 = nn.BatchNorm2d(in_channels)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_channels, bn_size * growth_rate, kernel_size=1, bias=False)
        
        self.norm2 = nn.BatchNorm2d(bn_size * growth_rate)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(bn_size * growth_rate, growth_rate, kernel_size=3, padding=1, bias=False)
        
        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = self.conv1(self.relu1(self.norm1(x)))
        new_features = self.conv2(self.relu2(self.norm2(new_features)))
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate, training=self.training)
        return torch.cat([x, new_features], 1)

class DenseBlock(nn.Module):
    def __init__(self, num_layers, in_channels, growth_rate, bn_size, drop_rate):
        super().__init__()
        layers = []
        for i in range(num_layers):
            layer = DenseLayer(
                in_channels + i * growth_rate,
                growth_rate,
                bn_size,
                drop_rate
            )
            layers.append(layer)
        self.block = nn.Sequential(*layers)

    def forward(self, x):
        return self.block(x)

class Transition(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.norm = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.conv(self.relu(self.norm(x)))
        return self.pool(x)

class DenseNetFromScratch(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.features = nn.Sequential()
        
        self.features.add_module('conv0', nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False))
        self.features.add_module('norm0', nn.BatchNorm2d(64))
        self.features.add_module('relu0', nn.ReLU(inplace=True))
        self.features.add_module('pool0', nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
        
        num_features = 64
        for i, num_layers in enumerate(config.block_config):
            block = DenseBlock(
                num_layers=num_layers,
                in_channels=num_features,
                growth_rate=config.growth_rate,
                bn_size=config.bn_size,
                drop_rate=config.drop_rate
            )
            self.features.add_module(f'denseblock{i+1}', block)
            num_features += num_layers * config.growth_rate
            
            if i != len(config.block_config)-1:
                trans = Transition(num_features, num_features // 2)
                self.features.add_module(f'transition{i+1}', trans)
                num_features = num_features // 2
        
        self.features.add_module('norm_final', nn.BatchNorm2d(num_features))
        self.features.add_module('relu_final', nn.ReLU(inplace=True))
        self.features.add_module('avg_pool', nn.AdaptiveAvgPool2d((1, 1)))
        self.classifier = nn.Linear(num_features, config.num_classes)

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        return self.classifier(x)

class HumanPoseDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.label_map = pd.read_csv(Config.categories_file).set_index('id')['category'].to_dict()
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_id = str(row['img_id']).strip()
        label = row['target_feature']
        img_path = os.path.join(self.img_dir, f"{img_id}.jpg")
        
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error loading {img_path}: {str(e)}")
            image = Image.new('RGB', Config.img_size)
            
        if self.transform:
            image = self.transform(image)
            
        return image, torch.tensor(label, dtype=torch.long)

def get_transforms(config, train=True):
    base_transforms = [
        transforms.Resize(config.img_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]
    
    if train:
        augmentations = [
            transforms.RandomHorizontalFlip(config.flip_prob),
            transforms.RandomRotation(15),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        ]
        return transforms.Compose(augmentations + base_transforms)
    return transforms.Compose(base_transforms)

def load_data(config):
    try:
        print("\n[1/4] Загрузка данных...")
        train_df = pd.read_csv(config.train_answers_file)
        print(f"Пример данных:\n{train_df.head()}")
        
        print("\n[2/4] Валидация данных...")
        train_df = train_df.dropna()
        print(f"Данных после очистки: {len(train_df)}")
        
        print("\n[3/4] Проверка изображений...")
        existing_images = {f.split('.')[0] for f in os.listdir(config.img_train_dir)}
        print(f"Найдено изображений: {len(existing_images)}")
        
        train_df = train_df[train_df['img_id'].astype(str).isin(existing_images)]
        print(f"Данных после фильтрации: {len(train_df)}")
        
        print("\n[4/4] Разделение данных...")
        train_df, test_val_df = train_test_split(
            train_df,
            test_size=config.test_size + config.val_size,
            stratify=train_df['target_feature'],
            random_state=config.random_state
        )
        val_df, test_df = train_test_split(
            test_val_df,
            test_size=config.test_size/(config.test_size + config.val_size),
            stratify=test_val_df['target_feature'],
            random_state=config.random_state
        )
        
        return (
            HumanPoseDataset(train_df, config.img_train_dir, get_transforms(config, True)),
            HumanPoseDataset(val_df, config.img_train_dir, get_transforms(config, False)),
            HumanPoseDataset(test_df, config.img_train_dir, get_transforms(config, False))
        )
    except Exception as e:
        print(f"\nData loading failed: {str(e)}")
        return None, None, None

class PoseTrainer:
    def __init__(self, config, device):
        self.config = config
        self.device = device
        self.best_metric = 0
        self.train_set, self.val_set, self.test_set = load_data(config)
        
        if not all([self.train_set, self.val_set, self.test_set]):
            raise RuntimeError("Data loading failed")
            
        self._init_dataloaders()
        self.model = DenseNetFromScratch(config).to(device)
        self.optimizer = torch.optim.AdamW(
            self.model.parameters(),
            lr=config.lr,
            weight_decay=config.weight_decay
        )
        self.scheduler = torch.optim.lr_scheduler.OneCycleLR(
            self.optimizer,
            max_lr=config.lr,
            epochs=config.epochs,
            steps_per_epoch=len(self.train_loader)
        )
        self.criterion = nn.CrossEntropyLoss()

    def _init_dataloaders(self):
        self.train_loader = DataLoader(
            self.train_set,
            batch_size=self.config.batch_size,
            shuffle=True,
            num_workers=self.config.num_workers,
            pin_memory=True
        )
        self.val_loader = DataLoader(
            self.val_set,
            batch_size=self.config.batch_size,
            num_workers=self.config.num_workers,
            pin_memory=True
        )
        self.test_loader = DataLoader(
            self.test_set,
            batch_size=self.config.batch_size,
            num_workers=self.config.num_workers,
            pin_memory=True
        )

    def train(self):
        os.makedirs(self.config.save_dir, exist_ok=True)
        history = {'train_loss': [], 'val_loss': [], 'val_acc': [], 'val_f1': []}
        
        for epoch in range(self.config.epochs):
            print(f"\nEpoch {epoch+1}/{self.config.epochs}")
            train_loss = self.train_epoch()
            val_metrics = self.evaluate(self.val_loader)
            
            history['train_loss'].append(train_loss)
            history['val_loss'].append(val_metrics['loss'])
            history['val_acc'].append(val_metrics['accuracy'])
            history['val_f1'].append(val_metrics['f1'])
            
            print(f"Train Loss: {train_loss:.4f}")
            print(f"Val Loss: {val_metrics['loss']:.4f}")
            print(f"Val Acc: {val_metrics['accuracy']:.4f}")
            print(f"Val F1: {val_metrics['f1']:.4f}")
            print(f"LR: {self.optimizer.param_groups[0]['lr']:.2e}")
            
            # Сохранение после каждой эпохи
            torch.save({
                'epoch': epoch,
                'model_state_dict': self.model.state_dict(),
                'optimizer_state_dict': self.optimizer.state_dict(),
                'metrics': val_metrics
            }, os.path.join(self.config.save_dir, f'checkpoint_epoch_{epoch+1}.pth'))
            
            if val_metrics['f1'] > self.best_metric:
                self.best_metric = val_metrics['f1']
                torch.save(self.model.state_dict(), 
                         os.path.join(self.config.save_dir, 'best_model.pth'))
        
        self.plot_training(history)
        return history

    def train_epoch(self):
        self.model.train()
        total_loss = 0
        all_preds = []
        all_labels = []
        
        with tqdm(
            self.train_loader, 
            desc='Training', 
            leave=False, 
            dynamic_ncols=True,
            unit='batch',
            bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}{postfix}]'
        ) as progress_bar:
            
            for images, labels in progress_bar:
                images = images.to(self.device)
                labels = labels.to(self.device)
                
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                self.scheduler.step()
                
                total_loss += loss.item()
                preds = torch.argmax(outputs, dim=1).cpu().numpy()
                all_preds.extend(preds)
                all_labels.extend(labels.cpu().numpy())
                
                batch_f1 = f1_score(labels.cpu().numpy(), preds, average='weighted')
                
                progress_bar.set_postfix({
                    'loss': f'{loss.item():.3f}',
                    'f1': f'{batch_f1:.3f}'
                })
        
        epoch_f1 = f1_score(all_labels, all_preds, average='weighted')
        return total_loss / len(self.train_loader), epoch_f1

    def evaluate(self, loader):
        self.model.eval()
        total_loss = 0
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for images, labels in loader:
                images = images.to(self.device)
                labels = labels.to(self.device)
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                total_loss += loss.item()
                all_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                
        return {
            'loss': total_loss / len(loader),
            'accuracy': np.mean(np.array(all_preds) == np.array(all_labels)),
            'f1': f1_score(all_labels, all_preds, average='weighted')
        }

    def plot_training(self, history):
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 3, 1)
        plt.plot(history['train_loss'], label='Train Loss')
        plt.plot(history['val_loss'], label='Val Loss')
        plt.title('Loss Evolution')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()

        plt.subplot(1, 3, 2)
        plt.plot(history['val_acc'], label='Validation Accuracy')
        plt.title('Accuracy Evolution')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')

        plt.subplot(1, 3, 3)
        plt.plot(history['val_f1'], label='Validation F1-Score')
        plt.title('F1-Score Evolution')
        plt.xlabel('Epoch')
        plt.ylabel('F1-Score')

        plt.tight_layout()
        plt.savefig(os.path.join(self.config.save_dir, 'training_progress.png'))
        plt.show()

if __name__ == "__main__":
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    config = Config()
    
    print("Path checks:")
    print("Categories file:", os.path.exists(config.categories_file))
    print("Train answers:", os.path.exists(config.train_answers_file))
    print("Image dir:", os.path.exists(config.img_train_dir))
    print(f"\nUsing device: {device}")
    
    try:
        trainer = PoseTrainer(config, device)
        history = trainer.train()
    except Exception as e:
        print(f"\nTraining failed: {e}")
        traceback.print_exc()

In [ ]:
import os
import torch
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
from tqdm import tqdm

# === Конфигурация ===
class Config:
    img_size = (224, 224)
    num_classes = 20
    batch_size = 32

    # Обновить пути, для тестирования модели

    test_dir = '??'
    checkpoint_path = '??'
    growth_rate = 12
    block_config = (4, 4, 4)
    bn_size = 4
    drop_rate = 0.2

class DenseLayer(nn.Module):
    def __init__(self, in_channels, growth_rate, bn_size, drop_rate):
        super().__init__()
        self.norm1 = nn.BatchNorm2d(in_channels)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_channels, bn_size * growth_rate, kernel_size=1, bias=False)
        self.norm2 = nn.BatchNorm2d(bn_size * growth_rate)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(bn_size * growth_rate, growth_rate, kernel_size=3, padding=1, bias=False)
        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = self.conv1(self.relu1(self.norm1(x)))
        new_features = self.conv2(self.relu2(self.norm2(new_features)))
        if self.drop_rate > 0:
            new_features = nn.functional.dropout(new_features, p=self.drop_rate, training=self.training)
        return torch.cat([x, new_features], 1)

class DenseBlock(nn.Module):
    def __init__(self, num_layers, in_channels, growth_rate, bn_size, drop_rate):
        super().__init__()
        layers = []
        for i in range(num_layers):
            layer = DenseLayer(
                in_channels + i * growth_rate,
                growth_rate,
                bn_size,
                drop_rate
            )
            layers.append(layer)
        self.block = nn.Sequential(*layers)

    def forward(self, x):
        return self.block(x)

class Transition(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.norm = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.conv(self.relu(self.norm(x)))
        return self.pool(x)

class DenseNetFromScratch(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.features = nn.Sequential()
        self.features.add_module('conv0', nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False))
        self.features.add_module('norm0', nn.BatchNorm2d(64))
        self.features.add_module('relu0', nn.ReLU(inplace=True))
        self.features.add_module('pool0', nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
        
        num_features = 64
        for i, num_layers in enumerate(config.block_config):
            block = DenseBlock(
                num_layers=num_layers,
                in_channels=num_features,
                growth_rate=config.growth_rate,
                bn_size=config.bn_size,
                drop_rate=config.drop_rate
            )
            self.features.add_module(f'denseblock{i+1}', block)
            num_features += num_layers * config.growth_rate
            
            if i != len(config.block_config)-1:
                trans = Transition(num_features, num_features // 2)
                self.features.add_module(f'transition{i+1}', trans)
                num_features = num_features // 2
        
        self.features.add_module('norm_final', nn.BatchNorm2d(num_features))
        self.features.add_module('relu_final', nn.ReLU(inplace=True))
        self.features.add_module('avg_pool', nn.AdaptiveAvgPool2d((1, 1)))
        self.classifier = nn.Linear(num_features, config.num_classes)

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        return self.classifier(x)

class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = sorted(
            [f for f in os.listdir(root_dir) if f.lower().endswith(('.jpg', '.png'))],
            key=lambda x: int(x.split('.')[0])
        )
        
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.root_dir, img_name)
        
        try:
            image = Image.open(img_path).convert('RGB')
        except:
            image = Image.new('RGB', Config.img_size)
            
        if self.transform:
            image = self.transform(image)
            
        return img_name, image

config = Config()
model = DenseNetFromScratch(config)

# Загрузка чекпоинта
checkpoint = torch.load(config.checkpoint_path, map_location='cpu')
model.load_state_dict(checkpoint['model_state_dict'], strict=True)
model.eval()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

test_transform = transforms.Compose([
    transforms.Resize(config.img_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_dataset = TestDataset(config.test_dir, transform=test_transform)
test_loader = DataLoader(
    test_dataset,
    batch_size=config.batch_size,
    shuffle=False,
    num_workers=0
)

predictions = []
image_ids = []

with torch.no_grad():
    for filenames, images in tqdm(test_loader, desc="Predicting"):
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())
        image_ids.extend(filenames)

# Формирование submission
submission_df = pd.DataFrame({
    'ID': [os.path.splitext(fname)[0] for fname in image_ids],
    'target': predictions
})

submission_df.to_csv('submission.csv', index=False)
print(f"Submission saved with {submission_df['target'].nunique()} unique classes")
